# Ciclo de vida
1. carga de datos
2. Ingeniería de características
3. Escalado de característiccas
4. Búsqueda de optimización de parámetros
5. Entrenamiento de K-Means
6. Análisis y visualización
7. Guardado del modelo
8. Predicción
   

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
# Guardado de modelo:
import joblib
from datetime import datetime
import warnings 

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_excel('./datasets/BD_Clientes_Productos.xlsx')
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299169 entries, 0 to 299168
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Fecha           299169 non-null  datetime64[ns]
 1   Nombre Cliente  299169 non-null  object        
 2   Tipo Cliente    299169 non-null  object        
 3   Departamento    299169 non-null  object        
 4   Vendedor        299169 non-null  object        
 5   Sucursal        299169 non-null  object        
 6   Categoría       299169 non-null  object        
 7   Producto        299169 non-null  object        
 8   Linea           299169 non-null  object        
 9   Cantidad        299169 non-null  int64         
 10  Venta           299169 non-null  float64       
 11  Costos          299169 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 27.4+ MB


In [4]:
df.head()

,Fecha,Nombre Cliente,Tipo Cliente,Departamento,Vendedor,Sucursal,Categoría,Producto,Linea,Cantidad,Venta,Costos
0,2018-01-04,OCEANO AZUL DISTRIBUCIONES S.A.S.,DISTRIBUIDOR,ATLANTICO,SILVIA CAROLINA LOPEZ,NORTE,LACTEA,CHEDDAR,QUESOS,5,45107.494665,41250.000000
1,2018-01-04,COMPAÑIA NACIONAL DE LEVADURAS LEVAPAN S.A.,CADENA REGIONAL,ANTIOQUIA,PAULA ANDREA ARTEAGA,CENTRO,LACTEA,MANTEQUILLA MINI,ESPARCIBLES,1,5717.000000,3712.872135
2,2018-01-04,RESTAURANTE JIANBANQ MEI,CADENA REGIONAL,ANTIOQUIA,PAULA ANDREA ARTEAGA,CENTRO,LACTEA,MANTEQUILLA MINI,ESPARCIBLES,1,5717.000000,3712.872135
3,2018-01-04,COMERCIALIZADORA Y ASOCIADOS S.A.S,CADENA REGIONAL,ANTIOQUIA,VICKY BUENAVENTURA,CENTRO,LACTEA,AREQUIPE MINI,POSTRES Y DULCES,1,8183.000000,4537.624369
4,2018-01-04,RAPI MERCAR SA,CADENA REGIONAL,ANTIOQUIA,LINA CECILIA CARDONA,CENTRO,LACTEA,MIGUELUCHO PER,POSTRES Y DULCES,1,8756.000000,5041.475267


In [5]:
# Cambiar el formato de fecha:
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')
df.head()

,Fecha,Nombre Cliente,Tipo Cliente,Departamento,Vendedor,Sucursal,Categoría,Producto,Linea,Cantidad,Venta,Costos
0,2018-01-04,OCEANO AZUL DISTRIBUCIONES S.A.S.,DISTRIBUIDOR,ATLANTICO,SILVIA CAROLINA LOPEZ,NORTE,LACTEA,CHEDDAR,QUESOS,5,45107.494665,41250.000000
1,2018-01-04,COMPAÑIA NACIONAL DE LEVADURAS LEVAPAN S.A.,CADENA REGIONAL,ANTIOQUIA,PAULA ANDREA ARTEAGA,CENTRO,LACTEA,MANTEQUILLA MINI,ESPARCIBLES,1,5717.000000,3712.872135
2,2018-01-04,RESTAURANTE JIANBANQ MEI,CADENA REGIONAL,ANTIOQUIA,PAULA ANDREA ARTEAGA,CENTRO,LACTEA,MANTEQUILLA MINI,ESPARCIBLES,1,5717.000000,3712.872135
3,2018-01-04,COMERCIALIZADORA Y ASOCIADOS S.A.S,CADENA REGIONAL,ANTIOQUIA,VICKY BUENAVENTURA,CENTRO,LACTEA,AREQUIPE MINI,POSTRES Y DULCES,1,8183.000000,4537.624369
4,2018-01-04,RAPI MERCAR SA,CADENA REGIONAL,ANTIOQUIA,LINA CECILIA CARDONA,CENTRO,LACTEA,MIGUELUCHO PER,POSTRES Y DULCES,1,8756.000000,5041.475267


In [6]:
snapshot_date = df['Fecha'].max() + pd.Timedelta(days=1)
# Agrupaciones por clientes: 
rfm_data = df.groupby('Nombre Cliente').agg({
    'Fecha': lambda date: (snapshot_date - date.max()).days, # Recencia
    'Nombre Cliente': 'count',                               # Frecuencia
    'Venta': 'sum'                                           # Valor Monetario
})

display(rfm_data)

,Fecha,Nombre Cliente,Venta
Nombre Cliente,,,
DISTRIBUIDORA PUNTO SEIS,2,68,1.417332e+06
5 ESQUINAS,46,50,9.394804e+05
61PRADO EUROPEAN GUESTHOUSE,2,209,3.135524e+06
ABARROTES ALFER,33,52,8.029471e+05
ABARROTES CUCUTA,39,54,7.615975e+05
...,...,...,...
ZEUS,72,11,1.812906e+05
ZONA LOGISTICA S.A.S,1,210,4.527208e+06
ZONA REFRESCANTE,72,11,1.090489e+05


In [7]:
# Renombramos las columnas
rfm_data.rename(columns = {
    'Fecha': 'Recencia', 
    'Nombre Cliente': 'Frecuencia',
    'Venta': 'Monetario'
}, inplace= True)

display(rfm_data)

,Recencia,Frecuencia,Monetario
Nombre Cliente,,,
DISTRIBUIDORA PUNTO SEIS,2,68,1.417332e+06
5 ESQUINAS,46,50,9.394804e+05
61PRADO EUROPEAN GUESTHOUSE,2,209,3.135524e+06
ABARROTES ALFER,33,52,8.029471e+05
ABARROTES CUCUTA,39,54,7.615975e+05
...,...,...,...
ZEUS,72,11,1.812906e+05
ZONA LOGISTICA S.A.S,1,210,4.527208e+06
ZONA REFRESCANTE,72,11,1.090489e+05


In [8]:
# Escalado de características:
scaler = StandardScaler()
rfm_scaler = scaler.fit_transform(rfm_data)

rfm_scaler_df = pd.DataFrame(rfm_scaler, index=rfm_data.index, columns=rfm_data.columns)
display(rfm_scaler_df)

,Recencia,Frecuencia,Monetario
Nombre Cliente,,,
DISTRIBUIDORA PUNTO SEIS,-1.401963,-0.113701,-0.098525
5 ESQUINAS,0.147700,-0.138627,-0.134109
61PRADO EUROPEAN GUESTHOUSE,-1.401963,0.081548,0.029422
ABARROTES ALFER,-0.310155,-0.135857,-0.144276
ABARROTES CUCUTA,-0.098838,-0.133088,-0.147355
...,...,...,...
ZEUS,1.063409,-0.192632,-0.190568
ZONA LOGISTICA S.A.S,-1.437182,0.082933,0.133056
ZONA REFRESCANTE,1.063409,-0.192632,-0.195948


## Método codo (WCSS)

In [ ]:
# Buscar el valor optimo de clusters:
wcss = {}
for k in range (2, len(rfm_scaler_df.index)):
    kmeans = KMeans(n_clusters = k, random_state=42, n_init=10, max_iter=300)
    kmeans.fit(rfm_scaler_df)
    wcss[k]= kmeans.inertia_

print(wcss.keys())

In [ ]:
# Gráfica de codo:
plt.figure(figsize(10,6))
plt.plot(list(wcss.keys()), list(wcss.values()), 'o-')
plt.title('Gráfica de codo para encontrar el k óptimo')
plt.xlabel('Cantidad de clusterd (K)')
plt.ylabel('WCSS . Inercia')
plt.grif(True)
plt.show()

In [ ]:
# Método de silueta:
silhoutte_score = {}
for k in range(2, len(rfm_scaler_df.index)):
    kmeans = KMeans(n_clusters = k, init='k-means++', n_init=10, random_state=42)
    kmeans.fit(rfm_scaler_df)
    score = silhoutte_scores(rfm_scaler_df, kmeans.labels_0)
    silhoutte_scores[k] = socre
    print(f'Coeficiente de silueta para k {k} = {score:.4f}')



In [11]:
K_OPTIMO = 3
# Entrenamiento final y generación del modelo k-means:
kmeans_final = KMeans(n_clusters = K_OPTIMO, init='k-means++', n_init=10, random_state=42)

kmeans_final.fit(rfm_scaler_df)

# Asignar la etiqueta del cluster a cada cliente:
rfm_data['Cluster'] = kmeans_final.labels_
print(f'Datos RFM con segmento asignado: ')
display(rfm_data)

Datos RFM con segmento asignado: 


,Recencia,Frecuencia,Monetario,Cluster
Nombre Cliente,,,,
DISTRIBUIDORA PUNTO SEIS,2,68,1.417332e+06,2
5 ESQUINAS,46,50,9.394804e+05,0
61PRADO EUROPEAN GUESTHOUSE,2,209,3.135524e+06,2
ABARROTES ALFER,33,52,8.029471e+05,0
ABARROTES CUCUTA,39,54,7.615975e+05,0
...,...,...,...,...
ZEUS,72,11,1.812906e+05,0
ZONA LOGISTICA S.A.S,1,210,4.527208e+06,2
ZONA REFRESCANTE,72,11,1.090489e+05,0


In [12]:
# Visualización de los segmentos:
cluster_analisis = rfm_data.groupby('Cluster').agg({
    'Recencia': 'mean',
    'Frecuencia': 'mean',
    'Monetario': ['mean', 'count']
}).round(2)
print(cluster_analisis)

        Recencia Frecuencia     Monetario      
            mean       mean          mean count
Cluster                                        
0          51.55      39.19  6.972857e+05  1597
1          12.66    5782.72  1.074426e+08    32
2           1.63     141.59  2.499759e+06   364


In [ ]:
# Figura de clusters:
fig = plt.figure(figsize(12,9))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(rfm_data['Recencia'])

In [13]:
# Guardado de modelo:
model_filename = './Models/best_kmeans_model.joblib'
scaler_filename = './Models/scaler.joblib'

joblib.dump(kmeans_final, model_filename)
joblib.dump(scaler, scaler_filename)

print(f'Modelo guardado en {model_filename}, scaler guardado en {scaler_filename}')

Modelo guardado en ./Models/best_kmeans_model.joblib, scaler guardado en ./Models/scaler.joblib


# Predicciones

In [15]:
model_filename = './Models/best_kmeans_model.joblib'
scaler_filename = './Models/scaler.joblib'
def predecir_segmento(datos_cliente):
    try:
        model = joblib.load(model_filename)
        scaler_model = joblib.load(scaler_filename)

        # Escalar los datos del cliente nuevo:
        datos_escalados = scaler_model.transform(datos_cliente)

        # Predicción:
        prediccion = model.predict(datos_escalados)
        return prediccion[0]

    except Exception as e:
        print(f'Se ha presentado fallas en la predicción, revise el error:\n{e}')

In [17]:
data_test = {
    'Recencia': [2],          # Compro hace 2 días
    'Frecuencia': [10],       # Ha realizado 10 compras
    'Monetario': [1000000]    # Ha gastado $1'000.000
}
data_df = pd.DataFrame(data_test)

In [18]:
# Predecir su segmento
segmento_predicho = predecir_segmento(data_df)

if segmento_predicho is not None:
    print(f"--- Prueba de la Función de Predicción ---")
    print(f"Datos del nuevo cliente:\n{data_df}")
    print(f"\nEl cliente pertenece al segmento: {segmento_predicho}")

--- Prueba de la Función de Predicción ---
Datos del nuevo cliente:
   Recencia  Frecuencia  Monetario
0         2          10    1000000

El cliente pertenece al segmento: 2
